<a href="https://colab.research.google.com/github/suryagokul/Data-Science-Portfolio/blob/master/Data-Science-Portfolio/Recommender%20Systems/Recommendation_Using_NearestNeighbours.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [274]:
import pandas as pd

import numpy as np

import sklearn

import seaborn as sns

In [275]:
movies_df = pd.read_csv("https://raw.githubusercontent.com/krishnaik06/Recommendation_complete_tutorial/master/KNN%20Movie%20Recommendation/movies.csv")

In [276]:
ratings_df = pd.read_csv("https://raw.githubusercontent.com/krishnaik06/Recommendation_complete_tutorial/master/KNN%20Movie%20Recommendation/ratings.csv")

In [277]:
movies_df[:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [278]:
ratings_df[:5]

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [279]:
ratings_df.drop('timestamp',axis=1,inplace=True)


In [280]:
ratings_df[:5]

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [281]:
df = pd.merge(movies_df,ratings_df,on='movieId')   #merging movies and ratings based on movieId

df[:5]

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5


In [282]:
df.isnull().any()

movieId    False
title      False
genres     False
userId     False
rating     False
dtype: bool

In [283]:
rating_count = pd.DataFrame(df.groupby('title')['rating'].count().reset_index().rename(columns={'rating':'no_of_ratings'}))

rating_count[:5]

,title,no_of_ratings
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [284]:
df = df.merge(rating_count,left_on='title',right_on='title',how='inner')

df.head()

,movieId,title,genres,userId,rating,no_of_ratings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,215
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,215
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,215
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,215
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,215


In [285]:
df[df['rating']>=4.5]['title'].unique()[:30]

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       'Father of the Bride Part II (1995)', 'Heat (1995)',
       'Sabrina (1995)', 'Tom and Huck (1995)', 'Sudden Death (1995)',
       'GoldenEye (1995)', 'American President, The (1995)',
       'Dracula: Dead and Loving It (1995)', 'Nixon (1995)',
       'Cutthroat Island (1995)', 'Casino (1995)',
       'Sense and Sensibility (1995)', 'Four Rooms (1995)',
       'Ace Ventura: When Nature Calls (1995)', 'Get Shorty (1995)',
       'Copycat (1995)', 'Assassins (1995)', 'Powder (1995)',
       'Leaving Las Vegas (1995)', 'Now and Then (1995)',
       'Persuasion (1995)',
       'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
       'Dangerous Minds (1995)',
       'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)', 'Babe (1995)',
       'Dead Man Walking (1995)', 'Clueless (1995)'], dtype=object)

In [286]:
final_df = df.pivot_table(index='title',columns='userId',values='rating').fillna(0)     

In [287]:
final_df.head()      # only showing ratings given by different users reamining all are given as zeros.. 

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [288]:
final_df[final_df.index=='Toy Story (1995)']

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0


In [289]:
from scipy.sparse import csr_matrix


In [290]:
final_df_matrix = csr_matrix(final_df.values)

In [291]:
from sklearn.neighbors import NearestNeighbors

In [292]:
NN = NearestNeighbors(algorithm='brute',metric='cosine')  # using Euclidean distsance and cosine similarity

In [293]:
NN.fit(final_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [333]:
random_movie = np.random.choice(final_df.shape[0])     # picking random movie and finding recommendations.

final_df.index[random_movie]

'Shakespeare in Love (1998)'

In [334]:
distances,indices = NN.kneighbors(final_df.iloc[random_movie,:].values.reshape(1,-1),n_neighbors=6)   # It gives distances and indices of that neighbours..

In [335]:
indices

array([[7565, 9381, 3282, 1025, 1346,  405]])

In [336]:
s = set()


In [337]:
for i in indices.flatten():
  s.add(final_df.index[i+1])

In [340]:
print("Recommendations for : ",final_df.index[random_movie])
print("")


Recommendations for :  Shakespeare in Love (1998)



In [341]:
d = {}


In [342]:
j=0
for i in distances.flatten():
  
  d[list(s)[j]] = i
  j+=1

In [343]:
d    # Distances from Shakespeare in Love (1998) movie.Lower the distance higher we recommende that movie...

{'American Buffalo (1996)': 0.4689243752968272,
 'Big Bang, The (2011)': 0.5096493292989197,
 "Breakfast at Tiffany's (1961)": 0.5236474112435874,
 'Full Moon in Blue Water (1988)': 0.5347119903755921,
 'Shall We Dance (1937)': 1.1102230246251565e-16,
 'When Night Is Falling (1995)': 0.5182240605776285}

In [344]:
f = pd.Series(d)

In [345]:
f = f.to_frame()  # Converting to dataframe.

In [346]:
f = f.reset_index()


In [347]:
f.columns=['Title',"Distance From "+final_df.index[random_movie]]

In [348]:
f      # Final Recommendation

,Title,Distance From Shakespeare in Love (1998)
0,Shall We Dance (1937),1.110223e-16
1,American Buffalo (1996),4.689244e-01
2,"Big Bang, The (2011)",5.096493e-01
3,When Night Is Falling (1995),5.182241e-01
4,Breakfast at Tiffany's (1961),5.236474e-01
5,Full Moon in Blue Water (1988),5.347120e-01
